In [1]:
import pandas as pd 
import pandas_datareader as web
import numpy as np


banks_data = pd.read_excel('Banks_Data.xlsx', sheet_name='Banks_Data') 

unique_tickers = banks_data.drop_duplicates(subset = ['ticker'])[['ticker', 'ShortName']]

def Dates(): 
    import datetime
    ## Keep the start and end dates for this experiment in an object so it can be accessed later on 

    start = datetime.date(2003, 1, 1)
    end = datetime.date(2015, 12, 31)
    return start, end

In [2]:
 def JoinTickersWithNames(sentiment_tickers, unique_tickers): 

        ## Turns the tickers into a dataframe with their names, then compares the name into the banks_data names. 

    tickers = pd.DataFrame.from_dict(sentiment_tickers, orient = 'index',
                                                                columns = ['YahooName']) 
    tickers.NewName = np.NaN
    tickers = tickers.join(unique_tickers.set_index('ticker') ) 
    return tickers

In [3]:
def CheckTickers(data): 
    from pandas_datareader._utils import RemoteDataError

    
    good_tickers = {}
    bad_tickers = {}
    tickers = []

    ## Return two dictionaries. One has all the tickers that work along with their full names
    ## One has the tickers that do not work

    

    for ticker in data.ticker: 
        try: 
            good_tickers[ticker] = web.get_quote_yahoo(ticker).shortName.tolist()[0]
            
        except KeyError: 
            bad_tickers[ticker] = 'KeyError'
            #print('KeyError in {}'.format(ticker) )

        except AttributeError: 
            bad_tickers[ticker] = 'AttributeError'
            #print('AttributeError in {}'.format(ticker))

        except RemoteDataError: 
            bad_tickers[ticker] = 'RemoteDataError- No data fetched'
            #print('Remote data error in {}'.format(ticker) ) 

        except IndexError: 
            bad_tickers[ticker] = 'IndexError'
            print(ticker) 
    
    return good_tickers, bad_tickers

In [ ]:
def Stocks(unique_tickers):  
    import pandas_datareader as web 
    ## Use previously defined functions to return data from stock market in the defined dates. If the ticker is valid, the data is returned. 
    ## If the ticker is not valid, it goes into the bad_tickers dataframe
    
    ## Create two dictionaries, one with ticker symbols that work, one with ticker symbols that do not. 
    good_tickers, bad_tickers = CheckTickers(unique_tickers) 

    ## Return Dates
    start, end = Dates()

    ## Return the Adjusted Closing Data for the tickers that work
    data = web.get_data_yahoo(list(good_tickers.keys()), start, end) 
    adjusted_close = data['Adj Close']
    close = data['Close']

    ## Calculate Log Returns
    log_returns = adjusted_close.apply(lambda x: np.log10(x)).diff().iloc[1:]





    ## Take the tickers and convert them to dataframe, compare the yahoo name with the given name from banks_data

   

    bad_tickers = JoinTickersWithNames(bad_tickers, unique_tickers) 
    good_tickers = JoinTickersWithNames(good_tickers, unique_tickers)





    return good_tickers, bad_tickers, log_returns, adjusted_close, close

good_tickers, bad_tickers, log_returns, adjusted_close, close = Stocks(unique_tickers) 

In [ ]:
def SplitGoodTickers(good_tickers, log_returns): 

    ## Split Stocks into dataframes of partial nulls, all nulls, or no nulls

    all_nulls = log_returns[log_returns.columns[log_returns.isna().all()]].columns.tolist()


    partial_nulls = log_returns[log_returns.columns[log_returns.isna().any()]].columns.tolist()
    for all_null_column in all_nulls: 
        partial_nulls.remove(all_null_column)

    no_nulls = log_returns[log_returns.columns[~log_returns.isna().any()]].columns.tolist()

    ## Anonymous Funtion to convert values to dataframe, join with the names of the tickers, as well as the adjusted closing prices of the tickers
    NullsToDataFrame = lambda x: pd.DataFrame(index = x).join(good_tickers).join(log_returns.transpose()).transpose()
    all_nulls = NullsToDataFrame(all_nulls) 
    partial_nulls = NullsToDataFrame(partial_nulls) 
    no_nulls = NullsToDataFrame(no_nulls) 

    return all_nulls, partial_nulls, no_nulls 

all_nulls, partial_nulls, no_nulls = SplitGoodTickers(good_tickers, log_returns) 

In [ ]:
with pd.ExcelWriter('Returns.xlsx') as writer: 
    banks_data.to_excel(writer, sheet_name = 'Banks_Data') 
    good_tickers.to_excel(writer, sheet_name = 'Good_Tickers')
    bad_tickers.to_excel(writer, sheet_name = 'Bad_Tickers') 
    all_nulls.to_excel(writer, sheet_name = 'All_Nulls') 
    partial_nulls.to_excel(writer, sheet_name = 'Partial_Nulls_Log_Returns') 
    no_nulls.to_excel(writer, sheet_name = 'No_Nulls_Log_Returns')
    log_returns.to_excel(writer, sheet_name = 'Log_Returns')
    adjusted_close.to_excel(writer, sheet_name = 'Adjusted_Closing_Prices')
    close.to_excel(writer, sheet_name = 'Closing_Prices')